In [160]:
import pandas as pd

df = pd.read_csv("../datasets/datasets/enron_spam_dataset.csv")

In [161]:
df.head(20)

,email_subject,email_body,is_spam
0,ena sales on hpl,just to update you on this project ' s status ...,0
1,98 - 6736 & 98 - 9638 for 1997 ( ua 4 issues ),the above referenced meters need to be placed ...,0
2,"hpl nominations for december 28 , 1999",( see attached file : hpll 228 . xls )\n- hpll...,0
3,revised nom - kcs resources,"daren ,\nit ' s in .\nbob\n- - - - - - - - - -...",0
4,new production - sitara deals needed,"daren ,\nfyi .\nbob\n- - - - - - - - - - - - -...",0
5,re : another hesco issue,help . steve mauch at hesco is wanting an answ...,0
6,"enron / hpl actuals for august 16 , 2000",teco tap 120 . 000 / hpl iferc ; 20 . 000 / en...,0
7,meter 1031 baytown exxon,daren - the valve for meter 1031 was not shut ...,0
8,ces deal clean - up,i will need to make these changes in sitara . ...,0
9,meter 5097,"daren ,\ndo you know if there should be a deal...",0


In [162]:
len(df)

33715

In [163]:
df.duplicated().sum()


np.int64(3222)

In [164]:
df.duplicated(subset=['email_body']).sum()


np.int64(3936)

In [165]:
df.isnull().sum()

email_subject    289
email_body       371
is_spam            0
dtype: int64

In [166]:
import re

def clean_text(s):
    s = str(s).lower()
    
    # Remove HTML tags first
    s = re.sub(r"<.*?>", " ", s)
    
    # Replace URLs, emails, and numbers with placeholders
    s = re.sub(r'http\S+', '<URL>', s)
    s = re.sub(r'\S+@\S+', '<EMAIL>', s)
    s = re.sub(r'\d+', '<NUM>', s)
    
    # Remove other punctuation but keep placeholders
    s = re.sub(r'[^a-zA-Z\s<>]', ' ', s)
    
    # Normalize spaces
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

# Apply to your DataFrame
df['clean'] = df['email_body'].apply(clean_text)




Placeholders:
Preserve meaningful patterns without overfitting, reduce the noise, and makes it easier to understand what the model is learning

In [167]:
df.sample(10)

,email_subject,email_body,is_spam,clean
25698,in the heart of your business !,corporate image can say a lot of things about ...,1,corporate image can say a lot of things about ...
26914,best deals on all generic viagra and generic c...,the thing is that a great errrection is provid...,1,the thing is that a great errrection is provid...
2540,galleryfurniture . com bowl,enron is proud to be a major sponsor of the ga...,0,enron is proud to be a major sponsor of the ga...
14528,fw : netco benefit rate projections,urgh !\n- - - - - original message - - - - -\n...,0,urgh original message from donnelly john l hr ...
32278,updates,dear client ! we added new o e n software .\nv...,1,dear client we added new o e n software vlslt ...
7932,re : information,"vince ,\nafter checking into this issue i have...",0,vince after checking into this issue i have fo...
13355,fw : releases,attached for your review are drafts of two new...,0,attached for your review are drafts of two new...
15793,need low priced software ? lemon tagging,"art learn company child half . clock red , up ...",1,art learn company child half clock red up nigh...
30749,preview our upcoming monthly hot pick before s...,wysak petroieum ( wysk )\ncurrent price : o . ...,1,wysak petroieum wysk current price o <NUM> apr...
10843,spamassassin . taint . org,"i discovered jmason . org in yahoo , my favori...",1,i discovered jmason org in yahoo my favorite d...


In [168]:
phish_df = df[df['is_spam'] == 1]
safe_df  = df[df['is_spam'] == 0]

phish_df.sample(20)

,email_subject,email_body,is_spam,clean
19325,become happy with your performance,"my girlfriend loves the results , but she does...",1,my girlfriend loves the results but she doesn ...
10377,perfect visual solution for your business now,working on your company ' s image ? start with...,1,working on your company s image start with a v...
9678,re : ink prices got you down ? 11956,would you like to\nsave up to 80 %\non printer...,1,would you like to save up to <NUM> on printer ...
30606,"congo clal 1 is , \ / 11 agrra , xanaax , adlp...","protocol totient tenure ,\nxanaax , \ / alium ...",1,protocol totient tenure xanaax alium cialiis i...
30934,re [ 7 ] : talk about our meds,skeller\noedwards\nfnoble\ntpedro\ntfuller\nae...,1,skeller oedwards fnoble tpedro tfuller aengels...
21510,slingshot,"barker ,\nvaliumxanaxcialis and _ more\nget hy...",1,barker valiumxanaxcialis and more get hydrocod...
3733,meet over 1 million girls sigletos,meta http - equiv = content - type content = t...,1,meta http equiv content type content text html...
15129,"the lowprice , great services or effective med...",superior qualities on permitted re - medies fo...,1,superior qualities on permitted re medies for ...
15367,uluslararasi telefon görÜÞmelerÝnde bÜyÜk tasa...,faturasiz\nÝetÝÞÝm\nuluslararasý telefon\ngörü...,1,faturasiz et m uluslararas telefon g r melerin...
25673,innovative and effective design for you !,adqueen is an institute committed\nto brand pl...,1,adqueen is an institute committed to brand pla...


In [169]:
df['is_spam'].value_counts(normalize=True)


is_spam
1    0.509269
0    0.490731
Name: proportion, dtype: float64

In [170]:
df.sample(10)


,email_subject,email_body,is_spam,clean
22231,lockdown your files,message loading\nimage not showing ? view mess...,1,message loading image not showing view message...
18272,gr,³ ? · s ? { ¦ µl\n³ ? · s ± µ ¦ ? 2\nµ @\n,1,s l s <NUM>
28419,cost of service announcement,please see attachment .,0,please see attachment
12904,fw : final documents,not sure what they have against canada ! anyho...,0,not sure what they have against canada anyhow ...
369,marshall ( hpl meter 6296 ) volume increase ef...,"daren ,\nis it already to change this nominati...",0,daren is it already to change this nomination ...
7117,ljm,vince / stinson :\nthe following is an update ...,0,vince stinson the following is an update on lj...
14121,structuring,earlier today mark breese resigned to accept a...,0,earlier today mark breese resigned to accept a...
16428,up and doing !,"take that !\noneh\nin helping others , we shal...",1,take that oneh in helping others we shall help...
6056,gene humphrey / mark palmer,vince :\ndo you have email addresses for gene ...,0,vince do you have email addresses for gene and...
27589,"all graphics software available , cheap oem ve...","good morning ,\nwe we offer latest oem package...",1,good morning we we offer latest oem packages o...


In [171]:
df.columns

Index(['email_subject', 'email_body', 'is_spam', 'clean'], dtype='object')

In [172]:
df = df.drop("email_body", axis=1)

In [173]:
df = df.drop("email_subject", axis=1)

In [174]:
df.sample(20)


,is_spam,clean
18962,1,la independent dec odefmx svoufdtywsaprpe lyri...
27726,0,michelle from our earlier conversation this is...
25817,1,this has been our closing notification we have...
30160,1,from the desk of the promotion manager shawsta...
213,0,spoke with ken reisz operations mgr and he sai...
160,0,hey fellas just for my records each of you guy...
3383,0,thank you for joining spinner com the web s la...
31686,1,checksummed oswald ray whirligig oersted bell ...
14762,0,greg action please call todd reid at duke back...
19551,1,black systems intel motherboard <NUM> fsb <NUM...


In [175]:
len(df)

33715

In [176]:
df.duplicated(subset=['clean']).sum()

np.int64(5665)

In [177]:
df = df.drop_duplicates(subset=['clean'])

delete dupes (rows with same body text)

In [178]:
len(df)

28050

In [179]:
df.isnull().sum()

is_spam    0
clean      0
dtype: int64

In [180]:
df.sample(20)

,is_spam,clean
1490,0,please find attached the <NUM> customer trip p...
13785,0,can i get a list of the traders that will make...
12066,0,hi louise to refresh your memory ena ooc took ...
13444,0,there are several elements which have been rel...
12191,0,louise this is the email written by paul biena...
7266,0,steve the journal of risk is a more technical ...
33267,1,looking for not expensive high quality softwar...
11470,0,enclosed is a revised draft of the two way isd...
8642,0,as a reminder the summer associate program is ...
795,0,i am pretty sure this is a high zero and will ...


Initial cleaning (clean column):
- Numbers, URLs, and emails are replaced with placeholders
- Lowercased text
- Removed HTML
- Removed punctuation
- Removed duplicate/nulls
- Normalized whitespace

In [181]:
df['is_spam'].value_counts(normalize=True)

is_spam
0    0.508734
1    0.491266
Name: proportion, dtype: float64

11/3/25: Finalizing DS cleaning 

In [182]:
# Check missing values
print("Missing values per column:")
print(df.isnull().sum())

# Quick sanity check of text content
print("\nExample cleaned text:")
print(df['clean'].head(5))

# Check label balance
print("\nLabel distribution:")
print(df['is_spam'].value_counts(normalize=True))


Missing values per column:
is_spam    0
clean      0
dtype: int64

Example cleaned text:
0    just to update you on this project s status ba...
1    the above referenced meters need to be placed ...
2      see attached file hpll <NUM> xls hpll <NUM> xls
3    daren it s in bob forwarded by robert cotten h...
4    daren fyi bob forwarded by robert cotten hou e...
Name: clean, dtype: object

Label distribution:
is_spam
0    0.508734
1    0.491266
Name: proportion, dtype: float64


In [183]:
import nltk
from nltk.corpus import stopwords,  words
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# download necessary data once
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # needed for lemmatizer
nltk.download('words')    # for english_vocab


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lle34\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lle34\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lle34\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lle34\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\lle34\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [184]:
import os

# Explicitly set NLTK's data directory to the one you downloaded to
nltk_data_dir = r"C:\Users\lle34\AppData\Roaming\nltk_data"
os.environ["NLTK_DATA"] = nltk_data_dir

# Add custom path to the search location
nltk.data.path.append(nltk_data_dir)

# Confirm
print("NLTK data paths:")
for p in nltk.data.path:
    print("  -", p)


NLTK data paths:
  - C:\Users\lle34/nltk_data
  - c:\Users\lle34\AppData\Local\Programs\Python\Python312\nltk_data
  - c:\Users\lle34\AppData\Local\Programs\Python\Python312\share\nltk_data
  - c:\Users\lle34\AppData\Local\Programs\Python\Python312\lib\nltk_data
  - C:\Users\lle34\AppData\Roaming\nltk_data
  - C:\nltk_data
  - D:\nltk_data
  - E:\nltk_data
  - C:\Users\lle34\AppData\Roaming\nltk_data
  - C:\Users\lle34\AppData\Roaming\nltk_data
  - C:\Users\lle34\AppData\Roaming\nltk_data
  - C:\Users\lle34\AppData\Roaming\nltk_data
  - C:\Users\lle34\AppData\Roaming\nltk_data


In [185]:
nltk.download('punkt_tab', download_dir='C:/Users/lle34/AppData/Roaming/nltk_data')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:/Users/lle34/AppData/Roaming/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [186]:
print("stopwords loaded:", len(stopwords.words('english')))
print("tokenizer test:", word_tokenize("hello world!"))

stopwords loaded: 198
tokenizer test: ['hello', 'world', '!']


In [187]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
placeholders = ['<URL>', '<EMAIL>', '<NUM>']

def preprocess_text_2(text, min_words=1):
    # Protect placeholders
    placeholder_map = {}
    for i, ph in enumerate(placeholders):
        temp_token = f'XPLACEHOLDERX{i}X'
        placeholder_map[temp_token.lower()] = ph.lower()
        text = re.sub(re.escape(ph), temp_token, text, flags=re.IGNORECASE)

    # Lowercase
    text = text.lower()

    # Tokenize
    tokens = word_tokenize(text)

    processed_tokens = []
    for word in tokens:
        # Restore placeholders
        if word in placeholder_map:
            processed_tokens.append(placeholder_map[word])
        elif word.isalpha() and word not in stop_words:
            processed_tokens.append(lemmatizer.lemmatize(word))

    if len(processed_tokens) < min_words:
        return None

    return " ".join(processed_tokens)


In [188]:
# Test examples
sample_texts = [
    "Check out <URL> or email me at <EMAIL>.",
    "The quick brown fox jumps over the lazy dogs.",
    "Claim your prize now! <NUM> clicks only."
]

for text in sample_texts:
    processed = preprocess_text_2(text)
    print("Processed:", processed or "Skipped garbled text")

Processed: check <url> email <email>
Processed: quick brown fox jump lazy dog
Processed: claim prize <num> click


- Tokenizes text: Splits the text into individual words (tokens) for processing.
- Removes stopwords
- Lemmatizes words: Reduces words to their base forms
- Protects placeholders from being removed

Produces..
- A cleaned, tokenized, lemmatized string of text.
- Keeps placeholders for feature recognition.
- Removes irrelevant or noisy words (stopwords, very short tokens).

In [189]:
df.sample(20)

,is_spam,clean
25738,1,get the finest rolex watch replica we only sel...
1307,0,i am pleased to announce that sabrae zajac wil...
32167,1,sa ve <NUM> <NUM> ord ering onl ine to day vi ...
9174,0,forwarded by kevin kindall corp enron on <NUM>...
12805,0,well due to calendar conflicts at utilicorp th...
14463,0,can you tell us on those where we avg what the...
22061,1,z <NUM> bt <NUM> at fraction of retail prices ...
13738,0,just to ensure that i am supplying you with th...
10225,1,exit list instructions pmoney
7657,0,shmuel this is the abstract for my presentatio...


The function takes a raw, cleaned email and produces a normalized version suitable for ML:

Lowercase, tokenized
Stopwords removed
Words lemmatized
Ready as one string per email

In [190]:
# Apply preprocessing to the 'clean' column
df['final_text'] = df['clean'].apply(preprocess_text_2)

# Quick check
df.head()


,is_spam,clean,final_text
0,0,just to update you on this project s status ba...,update project status based new report scott m...
1,0,the above referenced meters need to be placed ...,referenced meter need placed k please note inf...
2,0,see attached file hpll <NUM> xls hpll <NUM> xls,see attached file hpll <num> xl hpll <num> xl
3,0,daren it s in bob forwarded by robert cotten h...,daren bob forwarded robert cotten hou ect <num...
4,0,daren fyi bob forwarded by robert cotten hou e...,daren fyi bob forwarded robert cotten hou ect ...


Finalized text preprocessing/cleaning
- Stopwords removed
- Text tokenized
- Words lemmatized
- Emails normalized to a clean string ready for feature extraction

In [192]:
df.sample(20)

,is_spam,clean,final_text
24860,1,utahboysranch org has been assisting families ...,utahboysranch org assisting family need servic...
23292,0,sally thanks again for the positive feedback m...,sally thanks positive feedback member team ins...
10092,1,my girlfriend loves the results but she doesn ...,girlfriend love result know think natural thom...
29165,0,who knows if this is true mike triem na ipaq d...,know true mike triem na ipaq display business ...
4362,1,dear homeowner you have been pre approved for ...,dear homeowner pre approved <num> <num> home l...
15600,1,request for your unreserved asistance from bra...,request unreserved asistance branch manager un...
11809,0,louise as a result of the merlin buy back seve...,louise result merlin buy back several loan ref...
28813,0,i m not sure if this spreadsheet will work for...,sure spreadsheet work remember exact data want...
8814,0,dear all i change the storage model output to ...,dear change storage model output total value v...
9295,0,he told me that he has what he needs stinson v...,told need stinson vince j kaminski <num> <num>...


In [193]:
print("Missing values per column:")
print(df.isnull().sum())

Missing values per column:
is_spam       0
clean         0
final_text    5
dtype: int64


In [194]:
print("NaNs:", df['final_text'].isnull().sum())
print("Empty strings:", (df['final_text'].str.strip() == "").sum())


NaNs: 5
Empty strings: 0


Need to remove empty strings, also read as NaN

In [195]:
# Remove empty strings/whitespace-only rows
df = df[df['final_text'].str.strip() != ""].reset_index(drop=True)

# Remove NaN in final_text col
df = df.dropna(subset=['final_text'])



In [196]:
print(df.isnull().sum())


is_spam       0
clean         0
final_text    0
dtype: int64


In [197]:
df = df[['is_spam', 'final_text']]

In [198]:
#df.to_csv("enron_1.csv", index=False)